# Assignment 2 : link_prediction





## Download Package

In [1]:
# Install torch geometric
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 48 kB 2.2 MB/s 
     |████████████████████████████████| 407 kB 5.1 MB/s 


In [2]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 144 kB 45.8 MB/s 
     |████████████████████████████████| 181 kB 43.6 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=43f5cfa1712e52245581b4f3172493a7702444cf97065d51b971dc57cb025161
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [3]:
!pip install https://data.pyg.org/whl/torch-1.10.0%2Bcu113/torch_cluster-1.5.9-cp37-cp37m-linux_x86_64.whl



     |████████████████████████████████| 2.3 MB 2.2 MB/s 


## Import Package

In [4]:
import torch
import torch.nn as nn
from torch.nn import Linear
import csv
import numpy as np
from torch_geometric.data import Data
import wandb
from sklearn.metrics import f1_score
from torch_geometric.nn import GCNConv
import os
from pathlib import Path
import pandas as pd 


## dataset 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
d = 1
train1_path = f"/content/drive/My Drive/dataset/dataset{d}/train.csv"
test1_path = f"/content/drive/My Drive/dataset/dataset{d}/test.csv"
content_path = f"/content/drive/My Drive/dataset/dataset{d}/content.csv"
epocht= 5000


In [8]:
import sys

class load_data(): 
  def __init__(self, path, attr_path):
    super(load_data,self).__init__()    
    self.path = path
    self.attr_path = attr_path

  def read_data(self):
      file = pd.read_csv(self.path)
      label = file.loc[:,'label'].values
      bilabel = []
      bilabel.extend(label)
      bilabel.extend(label)
      bilabel= np.array(bilabel)
      edgeto = file.loc[:,'to'].values
      edgefrom = file.loc[:,'from'].values

      edge_index = [[],[]]
      pos_edge_index = [[],[]]
      neg_edge_index = [[],[]]
      non_zero = np.nonzero(np.array(label))[0]

      edge_index[0].extend(edgeto)
      edge_index[1].extend(edgefrom)
      edge_index[1].extend(edgeto)
      edge_index[0].extend(edgefrom)
      edge_index = torch.tensor(edge_index)
      # positive
      pos_edge_index[0].extend(edgeto[non_zero])
      pos_edge_index[1].extend(edgefrom[non_zero])
      pos_edge_index[1].extend(edgeto[non_zero])
      pos_edge_index[0].extend(edgefrom[non_zero])
      pos_edge_index = torch.tensor(pos_edge_index)

      # negative
      neg_edge_index[0].extend(edgeto[~non_zero])
      neg_edge_index[1].extend(edgefrom[~non_zero])
      neg_edge_index[1].extend(edgeto[~non_zero])
      neg_edge_index[0].extend(edgefrom[~non_zero])
      neg_edge_index = torch.tensor(pos_edge_index)

      return edge_index, pos_edge_index, neg_edge_index, bilabel


  def read_content(self):
      attribution = []; node=[]
      content = pd.read_csv(self.attr_path,delimiter='\t',header=None)
      contentArray = content.values
      attribute = contentArray[contentArray[:, 0].argsort()]
      np.set_printoptions(threshold=sys.maxsize)
      for i in attribute:
        n = i[0]
        x = i[1:]
        attribution.append(x)
        node.append(n)
      attribution = torch.tensor(attribution, dtype= torch.float)
      node_num = np.shape(node)[0]
      print(node_num)
      return attribution, node_num


In [9]:

Dt = load_data(train1_path,content_path)
edge_index, pos_edge, neg_edge,  label = Dt.read_data()
attribution, node_num = Dt.read_content()
data = Data(x=attribution, edge_index = edge_index, pos_edge_index=pos_edge, neg_edge_index=neg_edge, y =label, num_nodes = node_num)

attr = data.x
print(np.shape(attr[0])[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


2708
1433


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


# DeepWalk

In [ ]:
import networkx as nx
import random
import numpy as np
from typing import List
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec

class DeepWalk:
    def __init__(self, window_size: int, embedding_size: int, walk_length: int, walks_per_node: int):
        """
        :param window_size: window size for the Word2Vec model
        :param embedding_size: size of the final embedding
        :param walk_length: length of the walk
        :param walks_per_node: number of walks per node
        """
        self.window_size = window_size
        self.embedding_size = embedding_size
        self.walk_length = walk_length
        self.walk_per_node = walks_per_node

    def random_walk(self, g: nx.Graph, start: str, use_probabilities: bool = False) -> List[str]:
        """
        Generate a random walk starting on start
        :param g: Graph
        :param start: starting node for the random walk
        :param use_probabilities: if True take into account the weights assigned to each edge to select the next candidate
        :return:
        """
        walk = [start]
        for i in range(self.walk_length):
            neighbours = g.neighbors(walk[i])
            neighs = list(neighbours)
            if use_probabilities:
                probabilities = [g.get_edge_data(walk[i], neig)["weight"] for neig in neighs]
                sum_probabilities = sum(probabilities)
                probabilities = list(map(lambda t: t / sum_probabilities, probabilities))
                p = np.random.choice(neighs, p=probabilities)
            else:
                p = random.choice(neighs)
            walk.append(p)
        return walk

    def get_walks(self, g: nx.Graph, use_probabilities: bool = False) -> List[List[str]]:
        """
        Generate all the random walks
        :param g: Graph
        :param use_probabilities:
        :return:
        """
        random_walks = []
        for _ in range(self.walk_per_node):
            random_nodes = list(g.nodes)
            random.shuffle(random_nodes)
            for node in tqdm(random_nodes):
                random_walks.append(self.random_walk(g=g, start=node, use_probabilities=use_probabilities))
        return random_walks

    def compute_embeddings(self, walks: List[List[str]]):
        """
        Compute the node embeddings for the generated walks
        :param walks: List of walks
        :return:
        """
        model = Word2Vec(sentences=walks, window=self.window_size, vector_size=self.embedding_size)
        return model.wv

# Node2Vec

## first try

In [ ]:
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
from torch_sparse import SparseTensor
import torch_cluster  # noqa
from torch_geometric.utils.num_nodes import maybe_num_nodes

import torch_cluster  # noqa
random_walk = torch.ops.torch_cluster.random_walk
# except ImportError:
#     random_walk = None

EPS = 1e-15


class Node2Vec(torch.nn.Module):
    r"""The Node2Vec model from the
    `"node2vec: Scalable Feature Learning for Networks"
    <https://arxiv.org/abs/1607.00653>`_ paper where random walks of
    length :obj:`walk_length` are sampled in a given graph, and node embeddings
    are learned via negative sampling optimization.

    .. note::

        For an example of using Node2Vec, see `examples/node2vec.py
        <https://github.com/pyg-team/pytorch_geometric/blob/master/examples/
        node2vec.py>`_.

    Args:
        edge_index (LongTensor): The edge indices.
        embedding_dim (int): The size of each embedding vector.
        walk_length (int): The walk length.
        context_size (int): The actual context size which is considered for
            positive samples. This parameter increases the effective sampling
            rate by reusing samples across different source nodes.
        walks_per_node (int, optional): The number of walks to sample for each
            node. (default: :obj:`1`)
        p (float, optional): Likelihood of immediately revisiting a node in the
            walk. (default: :obj:`1`)
        q (float, optional): Control parameter to interpolate between
            breadth-first strategy and depth-first strategy (default: :obj:`1`)
        num_negative_samples (int, optional): The number of negative samples to
            use for each positive sample. (default: :obj:`1`)
        num_nodes (int, optional): The number of nodes. (default: :obj:`None`)
        sparse (bool, optional): If set to :obj:`True`, gradients w.r.t. to the
            weight matrix will be sparse. (default: :obj:`False`)
    """
    def __init__(self, edge_index, embedding_dim, walk_length, context_size,
                 walks_per_node=1, p=1, q=1, num_negative_samples=1,
                 num_nodes=None, sparse=False):
        super().__init__()

        # if random_walk is None:
        #     raise ImportError('`Node2Vec` requires `torch-cluster`.')

        N = maybe_num_nodes(edge_index, num_nodes)
        row, col = edge_index
        self.adj = SparseTensor(row=row, col=col, sparse_sizes=(N, N))
        self.adj = self.adj.to('cpu')

        assert walk_length >= context_size

        self.embedding_dim = embedding_dim
        self.walk_length = walk_length - 1
        self.context_size = context_size
        self.walks_per_node = walks_per_node
        self.p = p
        self.q = q
        self.num_negative_samples = num_negative_samples

        self.embedding = Embedding(N, embedding_dim, sparse=sparse)

        self.reset_parameters()

    def reset_parameters(self):
        self.embedding.reset_parameters()


    def forward(self, batch=None):
        """Returns the embeddings for the nodes in :obj:`batch`."""
        emb = self.embedding.weight
        return emb if batch is None else emb.index_select(0, batch)

    def loader(self, **kwargs):
        return DataLoader(range(self.adj.sparse_size(0)),
                          collate_fn=self.sample, **kwargs)


    def pos_sample(self, batch):
        batch = batch.repeat(self.walks_per_node)
        rowptr, col, _ = self.adj.csr()
        rw = random_walk(rowptr, col, batch, self.walk_length, self.p, self.q)
        if not isinstance(rw, torch.Tensor):
            rw = rw[0]

        walks = []
        num_walks_per_rw = 1 + self.walk_length + 1 - self.context_size
        for j in range(num_walks_per_rw):
            walks.append(rw[:, j:j + self.context_size])
        return torch.cat(walks, dim=0)


    def neg_sample(self, batch):
        batch = batch.repeat(self.walks_per_node * self.num_negative_samples)

        rw = torch.randint(self.adj.sparse_size(0),
                           (batch.size(0), self.walk_length))
        rw = torch.cat([batch.view(-1, 1), rw], dim=-1)

        walks = []
        num_walks_per_rw = 1 + self.walk_length + 1 - self.context_size
        for j in range(num_walks_per_rw):
            walks.append(rw[:, j:j + self.context_size])
        return torch.cat(walks, dim=0)


    def sample(self, batch):
        if not isinstance(batch, torch.Tensor):
            batch = torch.tensor(batch)
        return self.pos_sample(batch), self.neg_sample(batch)


    def loss(self, pos_rw, neg_rw):
        r"""Computes the loss given positive and negative random walks."""

        # Positive loss.
        start, rest = pos_rw[:, 0], pos_rw[:, 1:].contiguous()

        h_start = self.embedding(start).view(pos_rw.size(0), 1,
                                             self.embedding_dim)
        h_rest = self.embedding(rest.view(-1)).view(pos_rw.size(0), -1,
                                                    self.embedding_dim)

        out = (h_start * h_rest).sum(dim=-1).view(-1)
        pos_loss = -torch.log(torch.sigmoid(out) + EPS).mean()

        # Negative loss.
        start, rest = neg_rw[:, 0], neg_rw[:, 1:].contiguous()

        h_start = self.embedding(start).view(neg_rw.size(0), 1,
                                             self.embedding_dim)
        h_rest = self.embedding(rest.view(-1)).view(neg_rw.size(0), -1,
                                                    self.embedding_dim)

        out = (h_start * h_rest).sum(dim=-1).view(-1)
        neg_loss = -torch.log(1 - torch.sigmoid(out) + EPS).mean()

        return pos_loss + neg_loss


    def test(self, train_z, train_y, test_z, test_y, solver='lbfgs',
             multi_class='auto', *args, **kwargs):
        r"""Evaluates latent space quality via a logistic regression downstream
        task."""
        from sklearn.linear_model import LogisticRegression

        clf = LogisticRegression(solver=solver, multi_class=multi_class, *args,
                                 **kwargs).fit(train_z.detach().cpu().numpy(),
                                               train_y.detach().cpu().numpy())
        return clf.score(test_z.detach().cpu().numpy(),
                         test_y.detach().cpu().numpy())


    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.embedding.weight.size(0)}, '
                f'{self.embedding.weight.size(1)})')

In [ ]:
# get train test and val sizes: (70% - 15% - 15%)
train_size = int(data.num_nodes *0.7)
test_size = int(data.num_nodes*0.85) - train_size
val_size = data.num_nodes - train_size - test_size

In [ ]:
nodes = data.edge_index.t().numpy()
# get train test and validation set of nodes
train_set = nodes[0:train_size]
test_set = nodes[train_size:train_size+test_size]
val_set = nodes[train_size+test_size:]

print(len(train_set),len(test_set),len(val_set))
print(len(train_set)+len(test_set)+len(val_set) == len(nodes))

print("train set\t",train_set[:10])
print("test set \t",test_set[:10])
print("val set  \t",val_set[:10])

1895 406 15071
True
train set	 [[2399 2339]
 [2397 1144]
 [ 854 1726]
 [ 872  702]
 [2450 1312]
 [ 384 1277]
 [1808 2472]
 [  97 1861]
 [2030 2494]
 [ 682  100]]
test set 	 [[ 494  133]
 [ 485  260]
 [2114  561]
 [2092  913]
 [ 970 1128]
 [ 130 2541]
 [ 219  722]
 [ 692  122]
 [1473 2222]
 [1921 1337]]
val set  	 [[1772  199]
 [2563  394]
 [2420  761]
 [2474 1044]
 [2263  586]
 [  76 2685]
 [1002 2056]
 [1536 1676]
 [2067  114]
 [1993 2038]]


In [ ]:
print(len(nodes))

17372


In [ ]:
from torch_sparse.tensor import cpu
# build test train val masks
device = cpu
train_mask = torch.zeros(len(nodes),dtype=torch.long)
for i in train_set:
    train_mask[i] = 1.

test_mask = torch.zeros(len(nodes),dtype=torch.long)
for i in test_set:
    test_mask[i] = 1.
    
val_mask = torch.zeros(len(nodes),dtype=torch.long)
for i in val_set:
    val_mask[i] = 1.
    
print("train mask \t",train_mask[0:15])
print("test mask  \t",test_mask[0:15])
print("val mask   \t",val_mask[0:15]) 

train mask 	 tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0])
test mask  	 tensor([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])
val mask   	 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [ ]:
data.train_mask = train_mask
data.test_mask = test_mask
data.val_mask = val_mask

In [ ]:
# import torch_cluster
import os.path as osp
import torch
import torch_cluster
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm
from torch_geometric.datasets import TUDataset
import numpy as np
from sklearn.cluster import KMeans


In [ ]:
import torch_cluster  # noqa
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
from torch_sparse import SparseTensor

from torch_geometric.utils.num_nodes import maybe_num_nodes

random_walk = torch.ops.torch_cluster.random_walk

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20,
             context_size=10, walks_per_node=10,
             num_negative_samples=1, p=1, q=1, sparse=True).to(device)


loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:

# wandb.init(project="Link Prediction_ node2vec") 

def train():
    model.train()
    total_loss = 0
    time = 0 
    for pos_rw, neg_rw in loader:
        print(pos_rw, neg_rw)
        time +=1
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()                        
    return total_loss / len(loader)



for epoch in range(1, 200):
    min_valid_loss =2
    loss = train()
    #acc = test()
    if epoch % 20 == 0:    
      wandb.log({'pair_loss': loss, 'epoch': epoch})   
      print(f'epoch: {epoch} / training loss: {loss}')
      if min_valid_loss > loss:
        min_valid_loss = loss
      # Saving State Dict
        print("________checkpoint save model ______")
        torch.save(model.state_dict(), '/content/drive/My Drive/dataset/saved_mode_node2vec.pth')


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7ff8ba8d9d50>> (for pre_run_cell):


Exception: ignored

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


tensor([[ 441, 1995,  854,  ...,  563, 1840,  556],
        [ 521, 2415,  521,  ..., 1641, 1615, 2133],
        [1458,    1,  962,  ..., 1061, 2482, 1601],
        ...,
        [1074, 2187, 1643,  ...,   98, 1300, 1541],
        [ 274, 2463,  761,  ..., 2678, 1721, 1408],
        [ 243,  763,  982,  ...,  714, 1303,  714]]) tensor([[ 441, 2416, 1184,  ...,  930, 1618,  178],
        [ 521,  322,  943,  ..., 1706, 1824, 1557],
        [1458,  627, 1546,  ...,  508, 2634,  792],
        ...,
        [ 437, 2531, 2634,  ..., 1435, 1688, 1678],
        [1027, 2458, 1649,  ..., 1843, 2434,  365],
        [2417, 1692, 1216,  ..., 1361, 2541, 2266]])
tensor([[2692,  451, 1503,  ..., 2529,  993, 2529],
        [2615, 1691,  137,  ..., 1575, 2653, 1575],
        [2512,  781,  829,  ..., 1568,   81, 1568],
        ...,
        [1174,  144, 2624,  ..., 1314,  452,   45],
        [ 617,  209, 1649,  ..., 1151, 2116, 1151],
        [ 499,  330,  499,  ...,  232,  184, 2659]]) tensor([[2692,  992,  

KeyboardInterrupt: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7ff8ba8d9d50>> (for post_run_cell):


Exception: ignored

In [ ]:
@torch.no_grad()
def test():
  model.eval()
  z = model()
  # acc = model.test(z[data.train_mask], data.y[data.train_mask],z[data.test_mask], data.y[data.test_mask],max_iter=10)
  print(type(data.y))
  print(type(z))
  acc = model.test(z[data.train_mask], data.y[data.train_mask],z[data.test_mask], data.y[data.test_mask],max_iter=10)
                  
  score, eid = test()
  result = {"id": eid,
        "prob": acc}
  result = pd.DataFrame(result)
  result.to_csv(f'/content/drive/My Drive/dataset/result_node2vec{d}.csv',index=False)
  return acc
test()


<class 'numpy.ndarray'>
<class 'torch.nn.parameter.Parameter'>


AttributeError: ignored

## node2vecc

In [10]:
import multiprocessing
p = 1.0
q = 1.0
dimensions = 128
num_walks = 10
walk_length = 80
window_size = 10
num_iter = 1
workers = multiprocessing.cpu_count()

In [11]:
!pip install stellargraph

     |████████████████████████████████| 435 kB 5.0 MB/s 
     |████████████████████████████████| 462 kB 58.5 MB/s 


In [12]:
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec


def node2vec_embedding(graph, name):
    rw = BiasedRandomWalk(graph)
    walks = rw.run(graph.nodes(), n=num_walks, length=walk_length, p=p, q=q)
    print(f"Number of random walks for '{name}': {len(walks)}")

    model = Word2Vec(
        walks,
        size=dimensions,
        window=window_size,
        min_count=0,
        sg=1,
        workers=workers,
        iter=num_iter,
    )

    def get_embedding(u):
        return model.wv[u]

    return get_embedding

In [13]:
import networkx as nx
graph = nx.Graph()

In [14]:
#G.add_nodes_from(data.edge_index)
print(data.edge_index)

tensor([[2399, 2397,  854,  ..., 1711, 2440, 1222],
        [2339, 1144, 1726,  ...,  171,  633,  122]])


In [15]:

from stellargraph import StellarGraph
# print(data.edge_index[0])
# print(data.edge_index[1][1])
graph = nx.Graph()
graph.add_nodes_from(range(0, 2707))
# print(data.edge_index[0])
# for i in range(len(data.edge_index[0])):
  # print([data.edge_index[0][i], data.edge_index[1][i]])


In [17]:
print(len(data.edge_index[0]))

17372


In [37]:
np.array(np.shape(data.edge_index))
edge = np.array(data.edge_index)
print(edge[0][i], edge[1][i])

2399 2339


In [39]:
from stellargraph import StellarGraph
# print(data.edge_index[0])
# print(data.edge_index[1][1])
graph = nx.Graph()
graph.add_nodes_from(range(0, 2707))
print(graph)
for i in range(17372):
  graph.add_edge(edge[0][i], edge[1][i])
  # print([data.edge_index[0][i], data.edge_index[1][i]])
  # graph.add_edges_from([data.edge_index[0][i], data.edge_index[1][i]])
print(graph)
graph = graph.to_undirected()
# print(graph)
graph = StellarGraph.from_networkx(graph)


Graph with 2707 nodes and 0 edges
Graph with 2708 nodes and 8596 edges


In [41]:
from stellargraph.data import EdgeSplitter
from collections import Counter
import multiprocessing
from sklearn.model_selection import train_test_split
# Define an edge splitter on the original graph:

edge_splitter_test = EdgeSplitter(graph)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, examples_test, labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global"
)

print(graph_test.info())

** Sampled 859 positive and 859 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 7737

 Node types:
  default: [2708]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [7737]
        Weights: all 1 (default)
        Features: none


In [42]:
# Do the same process to compute a training subset from within the test graph
edge_splitter_train = EdgeSplitter(graph_test, graph)
graph_train, examples, labels = edge_splitter_train.train_test_split(
    p=0.1, method="global"
)
(
    examples_train,
    examples_model_selection,
    labels_train,
    labels_model_selection,
) = train_test_split(examples, labels, train_size=0.75, test_size=0.25)

print(graph_train.info())

** Sampled 773 positive and 773 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 6964

 Node types:
  default: [2708]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [6964]
        Weights: all 1 (default)
        Features: none


In [43]:
pd.DataFrame(
    [
        (
            "Training Set",
            len(examples_train),
            "Train Graph",
            "Test Graph",
            "Train the Link Classifier",
        ),
        (
            "Model Selection",
            len(examples_model_selection),
            "Train Graph",
            "Test Graph",
            "Select the best Link Classifier model",
        ),
        (
            "Test set",
            len(examples_test),
            "Test Graph",
            "Full Graph",
            "Evaluate the best Link Classifier",
        ),
    ],
    columns=("Split", "Number of Examples", "Hidden from", "Picked from", "Use"),
).set_index("Split")

,Number of Examples,Hidden from,Picked from,Use
Split,,,,
Training Set,1159,Train Graph,Test Graph,Train the Link Classifier
Model Selection,387,Train Graph,Test Graph,Select the best Link Classifier model
Test set,1718,Test Graph,Full Graph,Evaluate the best Link Classifier


In [44]:
p = 1.0
q = 1.0
dimensions = 128
num_walks = 10
walk_length = 80
window_size = 10
num_iter = 1
workers = multiprocessing.cpu_count()

In [46]:
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec


def node2vec_embedding(graph, name):
    rw = BiasedRandomWalk(graph)
    walks = rw.run(graph.nodes(), n=num_walks, length=walk_length, p=p, q=q)
    print(f"Number of random walks for '{name}': {len(walks)}")

    model = Word2Vec(
        walks,
        size=dimensions,
        window=window_size,
        min_count=0,
        sg=1,
        workers=workers,
        iter=num_iter,
    )

    def get_embedding(u):
        return model.wv[u]

    return get_embedding

In [ ]:
embedding_train = node2vec_embedding(graph_train, "Train Graph")

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler


# 1. link embeddings
def link_examples_to_features(link_examples, transform_node, binary_operator):
    return [
        binary_operator(transform_node(src), transform_node(dst))
        for src, dst in link_examples
    ]


# 2. training classifier
def train_link_prediction_model(
    link_examples, link_labels, get_embedding, binary_operator
):
    clf = link_prediction_classifier()
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    clf.fit(link_features, link_labels)
    return clf


def link_prediction_classifier(max_iter=2000):
    lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring="roc_auc", max_iter=max_iter)
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])


# 3. and 4. evaluate classifier
def evaluate_link_prediction_model(
    clf, link_examples_test, link_labels_test, get_embedding, binary_operator
):
    link_features_test = link_examples_to_features(
        link_examples_test, get_embedding, binary_operator
    )
    score = evaluate_roc_auc(clf, link_features_test, link_labels_test)
    return score


def evaluate_roc_auc(clf, link_features, link_labels):
    predicted = clf.predict_proba(link_features)

    # check which class corresponds to positive links
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(link_labels, predicted[:, positive_column])

In [ ]:
def operator_hadamard(u, v):
    return u * v


def operator_l1(u, v):
    return np.abs(u - v)


def operator_l2(u, v):
    return (u - v) ** 2


def operator_avg(u, v):
    return (u + v) / 2.0


def run_link_prediction(binary_operator):
    clf = train_link_prediction_model(
        examples_train, labels_train, embedding_train, binary_operator
    )
    score = evaluate_link_prediction_model(
        clf,
        examples_model_selection,
        labels_model_selection,
        embedding_train,
        binary_operator,
    )

    return {
        "classifier": clf,
        "binary_operator": binary_operator,
        "score": score,
    }


binary_operators = [operator_hadamard, operator_l1, operator_l2, operator_avg]

In [ ]:
results = [run_link_prediction(op) for op in binary_operators]
best_result = max(results, key=lambda result: result["score"])

print(f"Best result from '{best_result['binary_operator'].__name__}'")

pd.DataFrame(
    [(result["binary_operator"].__name__, result["score"]) for result in results],
    columns=("name", "ROC AUC score"),
).set_index("name")

# Model

In [ ]:

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)


    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        # cosine similarity
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim = -1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple = False).t()


In [ ]:
in_channel = np.shape(attr[0])[0]
model = Net(in_channel, 128, 1)
print(model)
# optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001, weight_decay=5e-4)
optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

Net(
  (conv1): GCNConv(1433, 128)
  (conv2): GCNConv(128, 1)
)


In [ ]:
total_examples = total_loss = 0
# wandb.watch(model)
# wandb.init(project="Link Prediction") 

def train():
    threshold = torch.tensor([0.4])
    min_valid_loss = 1
    model.train()
    for epoch in range(epocht):
          optimizer.zero_grad()
          z = model.encode(data.x, data.pos_edge_index)
          out = model.decode(z, data.edge_index).view(-1)
          s_predict = out.sigmoid()                   
          loss = criterion(out, torch.FloatTensor(data.y))
          predicted = (s_predict > threshold).float()
          f1score = f1_score(predicted.detach().numpy(), data.y.astype(float))
          loss.backward()
          optimizer.step()
          if epoch % 100 == 0:
            wandb.log({'pair_loss': loss, 'epoch': epoch})
            wandb.log({'f1score': f1score, 'epoch': epoch})
            print(f'epoch: {epoch} / training loss: {loss} /  f1score {f1score}')
            if min_valid_loss > loss:
              min_valid_loss = loss
            # Saving State Dict
              torch.save(model.state_dict(), '/content/drive/My Drive/dataset/saved_model.pth')

epoch,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇██
f1score,▁▁▅▆▇▇▇▇▇▇█████
pair_loss,█▇▄▃▃▂▂▂▂▂▁▁▁▁▁
epoch,1400
f1score,0.78124
pair_loss,0.43426


In [ ]:
train()

epoch: 0 / training loss: 0.6924976110458374 /  f1score 0.6647194465795542
epoch: 100 / training loss: 0.480236291885376 /  f1score 0.7647218453188602
epoch: 200 / training loss: 0.43853479623794556 /  f1score 0.7771759432200225
epoch: 300 / training loss: 0.4180329144001007 /  f1score 0.7830277830277831
epoch: 400 / training loss: 0.405199259519577 /  f1score 0.7834738936795224
epoch: 500 / training loss: 0.39701104164123535 /  f1score 0.7850218589621745
epoch: 600 / training loss: 0.39153310656547546 /  f1score 0.7876477316050323
epoch: 700 / training loss: 0.3872314989566803 /  f1score 0.7876993601375227
epoch: 800 / training loss: 0.38407137989997864 /  f1score 0.7872645568409982
epoch: 900 / training loss: 0.3818592429161072 /  f1score 0.7864374403056351
epoch: 1000 / training loss: 0.38200297951698303 /  f1score 0.7849164677804297
epoch: 1100 / training loss: 0.37731513381004333 /  f1score 0.7875489353575862
epoch: 1200 / training loss: 0.3771909773349762 /  f1score 0.78648027575

## Testing 

In [ ]:

class load_data():
    def __init__(self, ):
        super(load_data, self).__init__()
        self.data = {}

    def read_data(self,path):
        file = pd.read_csv(path)
        label = file.loc[:, 'label'].values
        edgeto = file.loc[:, 'to'].values
        edgefrom = file.loc[:, 'from'].values
        pos_edge_index = [[], []]
        non_zero = np.nonzero(np.array(label))[0]
        # positive
        pos_edge_index[0].extend(edgeto[non_zero])
        pos_edge_index[1].extend(edgefrom[non_zero])
        pos_edge_index[1].extend(edgeto[non_zero])
        pos_edge_index[0].extend(edgefrom[non_zero])
        edge_index = torch.tensor(pos_edge_index)
        return edge_index

    def read_test_data(self,path):
        file = pd.read_csv(path)
        edg_id = file.loc[:, 'id'].values
        edgeto = file.loc[:, 'to'].values
        edgefrom = file.loc[:, 'from'].values
        edge_index = [[], []]
        edge_index[0].extend(edgeto)
        edge_index[1].extend(edgefrom)
        test_edge_index = torch.tensor(edge_index)
        return test_edge_index, edg_id

    def read_content(self,path):
        attribution = [];
        node = []
        content = pd.read_csv(path, delimiter='\t', header=None)
        contentArray = content.values
        attribute = contentArray[contentArray[:, 0].argsort()]
        for i in attribute:
          n = i[0]
          x = i[1:]
          attribution.append(x)
          node.append(n)
        attribution = torch.tensor(attribution, dtype=torch.float)
        node_num = np.shape(node)[0]
        return attribution, node_num



In [ ]:
Dt = load_data()
pos_edge = Dt.read_data(train1_path)
edge_index_test, eid_test = Dt.read_test_data(test1_path)
attribution, node_num = Dt.read_content(content_path)

data = Data(x=attribution, edge_index=pos_edge, edge_index_test = edge_index_test, edge_id = eid_test )
edge_index= data.edge_index
attr = data.x
edge_index_test = data.edge_index_test
edge_id = data.edge_id


NameError: ignored

In [ ]:
in_channel = np.shape(attr[0])[0]
model = Net(in_channel, 128, 1)
model_path = torch.load('/content/drive/My Drive/dataset/saved_model.pth')
model.load_state_dict(model_path)

<All keys matched successfully>

In [ ]:
@torch.no_grad()
def test():
    model.eval()
    scores = []; eid = []
    z = model.encode(attr, edge_index)
    out = model.decode(z, edge_index_test).view(-1).sigmoid()
    scores.extend(out.numpy())
    eid.extend(np.array(edge_id))
    return scores, eid

In [ ]:

score, eid = test()
result = {"id": eid,
          "prob":score}
result = pd.DataFrame( result)
result.to_csv(f'result{d}.csv',index=False)